In [1]:
from pyspark import SparkContext

def createIndex(shapefile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(4326))
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

def findZone(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if zones.geometry[idx].contains(p):
            return idx

In [2]:
def getDrugsFromFileToSets(fn1,fn2):
    import re
    w=set()
    with open(fn1) as reader:
        rows = reader.read()
        for row in rows.split("\n"):
            w.add(" ".join(re.split(r'\W+',row)))
    with open(fn2) as reader:
        rows = reader.read()
        for row in rows.split("\n"):
            w.add(" ".join(re.split(r'\W+',row)))
    return w

In [7]:
def filterDrugs(tweets):
    import re
    if tweets[1]==0:
        return False
    drugs_set = broadcast_drug_set.value
    lst = re.split(r'\W+',tweets[2])
    lst = list(filter(lambda x:x!="",lst))
    i=0
    lst1=[]
    for word1 in lst:
        j=i+1
        temp =word1
        if(temp in drugs_set):
            return True
        for word2 in lst[j:]:
            temp = temp+" "+word2
            if(temp in drugs_set):
                return True
        i=i+1
    return False

In [8]:
def partition(tweetData):
    import pyproj
    import shapely.geometry as geom
    proj = pyproj.Proj(init="epsg:4326", preserve_units=False)
    main_index , main_zone = createIndex('500cities_tracts.geojson')
    city_index = None
    for tweet in tweetData:
        tweet_split = tweet.split('|')
        try:
            x = float(tweet_split[2])
            y = float(tweet_split[1])
            point = geom.Point((x,y))
            city_index = findZone(point,main_index,main_zone)
        except:
            pass
        if not city_index:
            pass
            #print(tweet_split[0])
        else:
            yield(main_zone['plctract10'][city_index],main_zone['plctrpop10'][city_index],tweet_split[5])

In [9]:
def mapper(data):
    return (data[0],data[1]),1
def normalizer(data):
    return data[0][0],float(data[1])/float(data[0][1])

In [10]:
if __name__=='__main__':
    #sc = SparkContext()
    main_index , main_zone = createIndex('500cities_tracts.geojson')
    broadcast_drug_set = sc.broadcast(getDrugsFromFileToSets('drug_illegal.txt','drug_sched2.txt'))
    broadcast_zone= sc.broadcast(main_zone)
    sc.textFile('/data/share/bdm/tweets-100m.csv').mapPartitions(partition).filter(filterDrugs).map(mapper).reduceByKey(lambda x,y:x+y).map(normalizer).sortBy(lambda x:x[0]).saveAsTextFile("final005")

In [7]:
a,b=createIndex('500cities_tracts.geojson')

In [8]:
b

,plctract10,plctrpop10,geometry
0,0107000-01073005910,4612,"(POLYGON ((-86.709181 33.617021, -86.707961 33..."
1,0107000-01073010801,168,"(POLYGON ((-86.74305200000001 33.515411, -86.7..."
2,0107000-01073012701,44,"(POLYGON ((-86.639171 33.525408, -86.639426 33..."
3,0107000-01073012703,498,"(POLYGON ((-86.628861 33.476501, -86.628784 33..."
4,0107000-01073012704,113,"(POLYGON ((-86.630489 33.472661, -86.630545 33..."
5,0135896-01073012704,27,"(POLYGON ((-86.617329 33.466873, -86.617450000..."
6,0135896-01073012907,2696,"(POLYGON ((-86.782253 33.386351, -86.7824 33.3..."
7,0135896-01073014409,3266,"(POLYGON ((-86.827608 33.376329, -86.827661000..."
8,0135896-01117030217,4878,"(POLYGON ((-86.628467 33.455782, -86.628476000..."
9,0135896-01117030303,2728,"(POLYGON ((-86.704643 33.425672, -86.704496000..."


In [9]:
import re
lst = re.split(u'\W+',"'I need more clothes 🙌🙌👌'")
lst

['', 'I', 'need', 'more', 'clothes', '']

In [10]:
i=0
lst1=[]
for word1 in lst:
    j=i+1
    temp =word1
    print(temp)
    for word2 in lst[j:]:
        temp = temp+" "+word2
        print(temp)
    i=i+1


 I
 I need
 I need more
 I need more clothes
 I need more clothes 
I
I need
I need more
I need more clothes
I need more clothes 
need
need more
need more clothes
need more clothes 
more
more clothes
more clothes 
clothes
clothes 



In [11]:
lst1

[]

In [12]:
filterDrugs("Well..eye opener I worked out. :")

l


False

In [13]:
"eye opener" in getDrugsFromFileToSets('drug_illegal.txt','drug_sched2.txt')
    

True

In [14]:
lst=[]
lst=[1,2,3]

In [15]:
lst

[1, 2, 3]